In [1]:
import sqlite3

# Connect to SQLite database (creates 'shop.db' if it doesn't exist)
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL
);

CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    total REAL NOT NULL,
    order_date TEXT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
''')

# Insert sample data
cursor.executemany('INSERT INTO users (name, email) VALUES (?, ?)', [
    ('Ajay Babu', 'ajay@gmail.com'),
    ('Pradeep Kumar', 'Pradeep@gmail.com')
])

cursor.executemany('INSERT INTO products (name, price) VALUES (?, ?)', [
    ('Laptop', 75000.00),
    ('Phone', 30000.00)
])

cursor.executemany('INSERT INTO orders (user_id, product_id, quantity, total, order_date) VALUES (?, ?, ?, ?, ?)', [
    (1, 1001, 5, 75000.00, '2025-08-18'),
    (2, 1002, 8, 60000.00, '2025-08-19')
])

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
# Query the shop.db
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()
for row in cursor.execute('SELECT * FROM orders'):
    print(row)
conn.close()

(1, 1, 1, 1, 75000.0, '2025-08-18')
(2, 2, 2, 2, 60000.0, '2025-08-19')


In [ ]:
#!pip install langchain_community
#!pip install -U langchain-google-genai
#!pip install langgraph.prebuilt
#!pip install langgraph

In [ ]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from langgraph.prebuilt import ToolNode
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
GEMENI_API_KEY = ''

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GEMENI_API_KEY)
db = SQLDatabase.from_uri("sqlite:///shop.db")

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [ ]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
)

In [18]:
question = "List all the users names?"

result = agent.invoke({"messages": [{"role": "user", "content": question}]})
print(result['messages'][-1].content)

The users names are Ajay Babu and Pradeep Kumar.
